# Getting familiar with the <span style="color:blue">Transit For London</span> API
The purpose of this project is to get familiar with complex lists and dictionaries, JSON files, and API requests using Python. In addition, we will learn how to work with API documentation. Information on how to use the <em> Transit For London </em> API can be found in the [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).

Each task below will help to learn how to develop the above mentioned skills, culminating with a final task: <em> planning the journey from Heathrow Airport to Tower Bridge using the bus and tube? Which way is faster? </em> 

For the final task, a short presentation of the findings will be made.

In [8]:
import requests as re
from IPython.display import JSON

### Adjusted custom get request function for the Transportation For London api- 
Prints response code and returns JSON tuple: regular and Jupyter-formatted

In [51]:
# Custom get function 
def get(url, endpoint):
    res = re.get(url + endpoint)
    json_r = res.json()
    json_f = JSON(json_r)
    
    print(res.status_code)
    
    return json_r, json_f

# Set the base url for all future get requests

url ='https://api.tfl.gov.uk/'

## Final Task
Plan the journey from Heathrow Airport to Tower Bridge using Bus and Tube? Which way is faster? Example output:
```
Planned duration:
Bus: x minutes
Tube: y minutes
```

We need to search the documentation for correct requests and parameters we need.

#### After looking at the documentation, I wrote a long string with the search endpoint for the required trip by <span style="color:blue">the Tube</span> from Heathrow to Tower Bridge

In [52]:
edpt = 'Journey/JourneyResults/Heathrow%20Airport/to/Tower%20Bridge?timeIs=Arriving&journeyPreference=LeastInterchange&mode=Tube'\
'&accessibilityPreference=NoRequirements&walkingSpeed=Slow&cyclePreference=None&bikeProficiency=Easy'

In [53]:
journey = get(url, endpoint=edpt)

300


#### Note: code <span style="color:blue">300</span> means there were multiple responses to request. I took a look at the keys to dictionary.

In [55]:
print(journey[0].keys())

dict_keys(['$type', 'toLocationDisambiguation', 'fromLocationDisambiguation', 'viaLocationDisambiguation', 'recommendedMaxAgeMinutes', 'searchCriteria', 'journeyVector'])


#### I examined the <span style="color:blue">formatted JSON</span> to know the request keys better to see how both <span style="color:red">toLocationDisambiguation</span> and <span style="color:red">fromLocationDisambiguation</span> are structured.

In [47]:
journey[1]

<IPython.core.display.JSON object>

#### After examining the dictionary, I decided to make a list of common place names near the Tower of London as there are multiple places for the destination

In [57]:
tower_vicinity = []
for location in journey[0]['toLocationDisambiguation']['disambiguationOptions']:
    tower_vicinity.append(location['place']['commonName'])
tower_vicinity

['City of London, Tower Bridge',
 'Southwark, One Tower Bridge',
 'Wapping (London), Tower Bridge Quay',
 'London Bridge, Tower Bridge Road',
 'Southwark, Tower Bridge Road',
 'Harrow (London), Tower Bridge Mews',
 'Southwark, Think Tower Bridge',
 'City of London, Tower Bridge Approach',
 'Tower Hamlets, Tower Bridge Exhibition',
 'London Bridge, Tower Bridge / City Hall',
 'Southwark, Draft House Tower Bridge',
 'Tower Hamlets [Stepney], Grange Tower Bridge Hotel',
 'City of Westminster, Novotel Tower Bridge Hotel',
 'Tower Hamlets [Stepney], Try Tag Rugby Tower Bridge',
 'Southwark, Tower Bridge Magistrates Court',
 'Southwark, Chicken Cottage Takeaway Food, Tower Bridge']

#### I did the same for the departure point as there are multiple places to depart the airport from

In [212]:
airport = []
for location in journey[0]['fromLocationDisambiguation']['disambiguationOptions']:
    airport.append(location['place']['commonName'])
airport

['Hillingdon, Heathrow Airport Terminal 2',
 'Heathrow Airport, Heathrow Airport Terminal 3',
 'Hillingdon, London Heathrow Airport Terminal 5',
 'Hillingdon, Heathrow Airport Terminal 5 Arrivals',
 'Hillingdon, Heathrow Airport Terminal 4 Arrivals',
 'Hillingdon, Heathrow Airport Terminal 3 Arrivals',
 'Hillingdon, United Airlines, Heathrow Airport Terminal 2',
 'Heathrow Airport Terminals 1-3, Heathrow Airport Terminals 2 & 3 Rail Station']

### I made a dictionary with each place (departure and arrival) and the latitude and longitude of each location

In [58]:
tower_vicinity_ll = {}
for location in journey[0]['toLocationDisambiguation']['disambiguationOptions']:
    tower_vicinity_ll[location['place']['commonName']] = (location['place']['lat'],location['place']['lon'])
print(tower_vicinity_ll)

{'City of London, Tower Bridge': (51.50599630145, -0.07502752220999999), 'Southwark, One Tower Bridge': (51.503958003700006, -0.07740449013999999), 'Wapping (London), Tower Bridge Quay': (51.50622947685, -0.07334614114), 'London Bridge, Tower Bridge Road': (51.50227892501, -0.0786566734), 'Southwark, Tower Bridge Road': (51.49743849977, -0.08015688257), 'Harrow (London), Tower Bridge Mews': (51.55796833405, -0.3351140985), 'Southwark, Think Tower Bridge': (51.500226935220006, -0.07910317811999999), 'City of London, Tower Bridge Approach': (51.508139082110006, -0.07408698432000001), 'Tower Hamlets, Tower Bridge Exhibition': (51.50551623644, -0.07536476862000001), 'London Bridge, Tower Bridge / City Hall': (51.50354961675, -0.07662917924), 'Southwark, Draft House Tower Bridge': (51.502917014030004, -0.07698722055), 'Tower Hamlets [Stepney], Grange Tower Bridge Hotel': (51.511481522249994, -0.07227427369), 'City of Westminster, Novotel Tower Bridge Hotel': (51.51092054902, -0.0775437131),

In [59]:
airport_ll = {}
for location in journey[0]['fromLocationDisambiguation']['disambiguationOptions']:
    airport_ll[location['place']['commonName']] = (location['place']['lat'],location['place']['lon'])
print(airport_ll)

{'Hillingdon, Heathrow Airport Terminal 2': (51.46962511264, -0.44968473098), 'Heathrow Airport, Heathrow Airport Terminal 3': (51.47254839367, -0.45724573875), 'Hillingdon, London Heathrow Airport Terminal 5': (51.47238603683, -0.48688352825), 'Hillingdon, Heathrow Airport Terminal 5 Arrivals': (51.47212537413, -0.48689215164), 'Hillingdon, Heathrow Airport Terminal 4 Arrivals': (51.46046306549, -0.44636767138), 'Hillingdon, Heathrow Airport Terminal 3 Arrivals': (51.471544550269996, -0.45886341771), 'Hillingdon, United Airlines, Heathrow Airport Terminal 2': (51.4697660699, -0.44946398635999996), 'Heathrow Airport Terminals 1-3, Heathrow Airport Terminals 2 & 3 Rail Station': (51.47198549429, -0.45478818469000004)}


## After looking at various locations in the tower vicinity, I decided to make my presentation from the perspective of the Novotel Tower Bridge Hotel

In [185]:
# Confirm the hotel name and location
novotel_tower_bridge = journey[0]['toLocationDisambiguation']['disambiguationOptions'][12]['place']
name = novotel_tower_bridge['commonName']
lat = novotel_tower_bridge['lat']
long = novotel_tower_bridge['lon']
print(name)
print(f'lat: {lat}, long: {long}')

City of Westminster, Novotel Tower Bridge Hotel
lat: 51.51092054902, long: -0.0775437131


## After looking at Heathrow Airport, I decided the basic departure point from the airport to be the Rail Station at the final index, and at Terminal 4 & 5, which are in separte locations and a bit of a distance from the terminals 1 and 3 (terminal 1-3 are in the centre, terminal 4 is in the south, and terminal 5 in the west side of the airport)

#### Get the GPS coordinates for Novotel Tower Bridge Hotel and Airport Rail Station to use in my URL request

In [207]:
novotel = tower_vicinity[12]
terminal = airport[7]
lat_1 = str(airport_ll[terminal][0])
long_1 = str(airport_ll[terminal][1])

lat_2 = str(tower_vicinity_ll[novotel][0])
long_2 = str(tower_vicinity_ll[novotel][1])
lat_1, long_1, lat_2, long_2

('51.47198549429', '-0.45478818469000004', '51.51092054902', '-0.0775437131')

#### Build the URL manually to use in custom function below

In [294]:
url='https://api.tfl.gov.uk/Journey/JourneyResults/51.47198549429%2C-0.45478818469000004/to/51.51092054902%2C-0.0775437131?'

# default the url from Heathrow Airport Terminals 2 & 3 Rail Station
def get_to_novotel(url, date, time , going, transport):
    pref = 'LeastTime'
    res = re.get(url, params={'date': date, 'time': time, 'timeIs': going, 'mode': transport, 'journeyPreference': pref})
    json_r = res.json()
    json_f = JSON(json_r)
    code = res.status_code
    return json_r, json_f, code

In [295]:
terminal = airport[3]
lat_1 = str(airport_ll[terminal][0])
long_1 = str(airport_ll[terminal][1])
lat_1, long_1, lat_2, long_2

('51.47212537413', '-0.48689215164', '51.51092054902', '-0.0775437131')

In [297]:
morning_tube = get_to_novotel(url, '20220607', '0900', 'Departing', 'tube') 
morning_tube[1]

<IPython.core.display.JSON object>

In [299]:
noon_tube = get_to_novotel(url, '20220607', '1200', 'Departing', 'tube') 
noon_tube[1]

<IPython.core.display.JSON object>

In [315]:
def directions(route):
    start = route['journeys'][0]['startDateTime'][-8:-3]   
    stop = route['journeys'][0]['arrivalDateTime'][-8:-3] 
    departure = route['journeys'][0]['legs'][1]['departureTime'][-8:-3]
    duration = route['journeys'][0]['duration']
    print(f'Your start time from the Terminals 1-3 is: {start}.' )
    print(f'Your arrival time at the Novotel London Tower Bridge is: {stop}')
    print(f'Duration: {duration} minutes')
    print('\nDirections:')
    print(route['journeys'][0]['legs'][0]['instruction']['detailed'])
    print(route['journeys'][0]['legs'][1]['instruction']['detailed'], end="")
    print(f' -> Train departs at {departure}')
    print(route['journeys'][0]['legs'][2]['instruction']['detailed'])
    print(route['journeys'][0]['legs'][3]['instruction']['detailed'])
    
# directions(route)    

In [316]:
morning_route =  morning_tube[0]
directions(morning_route)

Your start time from the Terminals 1-3 is: 08:59.
Your arrival time at the Novotel London Tower Bridge is: 10:14
Duration: 75 minutes

Directions:
Walk to Heathrow Terminals 2 & 3 Underground Station
Piccadilly line towards Arnos Grove -> Train departs at 09:07
District line towards Upminster
Walk to 25 Savage Gardens, City


In [317]:
noon_route =  noon_tube[0]
directions(noon_route)

Your start time from the Terminals 1-3 is: 11:59.
Your arrival time at the Novotel London Tower Bridge is: 13:13
Duration: 74 minutes

Directions:
Walk to Heathrow Terminals 2 & 3 Underground Station
Piccadilly line towards Cockfosters -> Train departs at 12:07
District line towards Upminster
Walk to 25 Savage Gardens, City


### Build the URL using two custom functions

In [226]:
def departure_arrival_gps(departure_index, arrival_index):
    departure_point = airport[departure_index]
    destination = tower_vicinity[arrival_index]
    
    lat_departure = str(airport_ll[departure_point][0])
    long_departure = str(airport_ll[departure_point][1])

    lat_arrival = str(tower_vicinity_ll[destination][0])
    long_arrival = str(tower_vicinity_ll[destination][1])
    
    return lat_departure, long_departure, lat_arrival, long_arrival

def build_url(base_url, endpoint, departure_index, arrival_index):
    lat_1, long_1, lat_2, long_2 = departure_arrival_gps(departure_index, arrival_index)
    
    ll_ep = lat_1 + '%2C' + long_1 + '/to/' + lat_2 + '%2C' + long_2 + '?'
    final_url = base + endpoint + ll_ep
    
    return final_url

### Use index 3 (Heathrow Terminal 5 Arrivals) and 12 (Nototel Tower Bridge)

In [319]:
default_url = build_url(base_url='https://api.tfl.gov.uk/', endpoint='Journey/JourneyResults/', departure_index=3, arrival_index=12)
# Confrim URL is correcdt
print(url)

https://api.tfl.gov.uk/Journey/JourneyResults/51.47212537413%2C-0.48689215164/to/51.51092054902%2C-0.0775437131?


In [330]:
# default the url from Heathrow Airport Terminals 5 Arrivals as built above
def get_to_novotel_5(date, time , going, transport, url=default_url):
    pref = 'LeastTime'
    res = re.get(url, params={'date': date, 'time': time, 'timeIs': going, 'mode': transport, 'journeyPreference': pref})
    json_r = res.json()
    json_f = JSON(json_r)
    code = res.status_code
    return json_r, json_f, code

In [331]:
morning_tube_5 = get_to_novotel_5('20220607', '0900', 'Departing', 'tube')
morning_tube_5[1]

<IPython.core.display.JSON object>

In [332]:
afternoon_tube_5 = get_to_novotel_5('20220607', '1300', 'Departing', 'tube')
afternoon_tube_5[1]

<IPython.core.display.JSON object>

In [333]:
def directions_5(route):
    route = route['journeys'][0]
    start = route['startDateTime'][-8:-3]  
    stop = route['arrivalDateTime'] [-8:-3]
    print(f'Your start time from the Terminal 5 is: {start}' )
    print(f'Your arrival time at the Novotel London Tower Bridge is: {stop}')
    print('\nDirections:')
    steps = len(route['legs'])
    for i in range(steps):
        print(str(i+1) + ") ", end="")
        
        if i == 0:
            print("Walk to " + str(route['legs'][i]['arrivalPoint']['commonName'])) 
#             print(str(route['legs'][i]['instruction']['summary']) + str(route['legs'][i]['arrivalPoint']['commonName']) ) 
#         elif i == (steps - 1):
#             print(route['legs'][i]['instruction']['summary'])
        elif (i != 0) and (i != steps - 1):
            print("Take ", end="")
            print(route['legs'][i]['instruction']['summary'])
        else:
            print(route['legs'][i]['instruction']['summary'])
        
    fare = route['fare']['totalCost']
    duration = route['duration']

    print(f'\nThe total trip time is approximately: {duration} minutes')
    print(f'\nThe cost of this trip is: £{fare} ')

In [334]:
morning_route_5 = morning_tube_5[0]
directions2(morning_route_5)

Your start time from the Terminal 5 is: 08:59
Your arrival time at the Novotel London Tower Bridge is: 10:14

Directions:
1) Walk to Heathrow Terminal 5 Underground Station
2) Take Piccadilly line to Hammersmith (Dist&Picc Line)
3) Take District line to Tower Hill
4) Walk to 25 Savage Gardens, City

The total trip time is approximately: 75 minutes

The cost of this trip is: £550 


In [336]:
afternoon_route_5 = afternoon_tube_5[0]
directions_5(afternoon_route_5)

Your start time from the Terminal 5 is: 12:59
Your arrival time at the Novotel London Tower Bridge is: 14:13

Directions:
1) Walk to Heathrow Terminal 5 Underground Station
2) Take Piccadilly line to Hammersmith (Dist&Picc Line)
3) Take District line to Tower Hill
4) Walk to 25 Savage Gardens, City

The total trip time is approximately: 74 minutes

The cost of this trip is: £350 


In [339]:
morning_bus = get_to_novotel(url, '20220607', '0900', 'Departing', 'bus')
morning_bus[1]

<IPython.core.display.JSON object>

In [343]:
directions(morning_bus[0])

Your start time from the Terminals 1-3 is: 08:03.
Your arrival time at the Novotel London Tower Bridge is: 11:15
Duration: 192 minutes

Directions:
Walk to Heathrow Terminal 5
RA2 bus towards Heathrow Central Bus Stn -> Train departs at 08:05
X140 bus towards Harrow, Limited Stop
607 bus towards White City, Limited Stop


In [345]:
noon_bus = get_to_novotel(url, '20220607', '1200', 'Departing', 'bus')
noon_bus[1]

<IPython.core.display.JSON object>

In [346]:
directions(noon_bus[0])

Your start time from the Terminals 1-3 is: 11:58.
Your arrival time at the Novotel London Tower Bridge is: 14:57
Duration: 179 minutes

Directions:
Walk to Heathrow Terminal 5
482 bus towards Southall, Town Hall -> Train departs at 11:59
H91 bus towards Hammersmith
9 bus towards Aldwych


#### Let's check what other modes of transport are available from the airport

In [352]:
modes = get(base, endpoint='Line/Meta/Modes')

200


In [354]:
types = []
other_types = []
for mode in modes[0]: 
    name = mode['modeName']
    if mode['isTflService']:
        types.append(name)
    else:
        other_types.append(mode['modeName'])

In [356]:
for i in types:
    a = get_to_novotel(url, '20220607', '1500', 'Departing', i)[2] 
    print(i, a)

bus 200
cable-car 404
cycle-hire 200
dlr 404
elizabeth-line 404
overground 404
replacement-bus 404
river-bus 404
river-tour 200
tflrail 404
tram 404
tube 200


In [358]:
morning_river_tour = get_to_novotel(url, '20220607', '0900', 'Departing', 'river-tour') 
morning_river_tour[1]

<IPython.core.display.JSON object>

In [359]:
from datetime import datetime

datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')

In [360]:
arrival = datetime.strptime(morning_river_tour[0]['journeys'][0]['arrivalDateTime'], '%Y-%m-%dT%H:%M:%S')
start = datetime.strptime(morning_river_tour[0]['journeys'][0]['startDateTime'], '%Y-%m-%dT%H:%M:%S')
time = arrival - start
time.seconds/60

65.0

In [361]:
directions(morning_river_tour[0])

Your start time from the Terminals 1-3 is: 08:51.
Your arrival time at the Novotel London Tower Bridge is: 09:56
Duration: 65 minutes

Directions:
Walk to Heathrow Terminal 5
Heathrow Express towards London Paddington -> Train departs at 08:57
Walk to Paddington Underground Station
Circle line towards High Street Kensington and Victoria


In [133]:
terminal_2 = get(endpoint='Journey/JourneyResults/51.46962511264%2C-0.44968473098/to/51.50599630145%2C-0.07502752220999999?date=20211125&time=0900&timeIs=Departing')

TypeError: get() missing 1 required positional argument: 'url'

In [ ]:
terminal_2[2]

In [ ]:
JSON(t2.json())

In [ ]:
t3 = re.get('https://api.tfl.gov.uk/Journey/JourneyResults/51.46962511264%2C-0.44968473098/to/51.50599630145%2C-0.07502752220999999?mode=tube')

In [ ]:
JSON(t3.json())